In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
from IPython.display import clear_output

In [0]:
!pip install pycodestyle flake8 pycodestyle_magic
%load_ext pycodestyle_magic
clear_output()

In [0]:
!pip install natasha
clear_output()

In [0]:
!pip install joblib
clear_output()

Задача: научиться находить в тексте лексически/стилистически неправильные сочетания (коллокации) и предлагать более правильные замены. <br>
Алгоритм является улучшенной версией этого алгоритма https://github.com/annadmitrieva/NLP-stuff/blob/master/collocation_generation-Copy4.ipynb 

#Этапы работы

*   Автоматическое обнаружение "неправильных" коллокаций в тексте.
*   Поиск замен.

Скачиваем word2vec модель из rusvectores.

In [0]:
!wget https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz
clear_output()

Для определения домена текста используем предобученную на шестиграммах из каждого домена (по 100000 на домен) SVM. F1-score модели – 0.94. </br>
Код модели: https://github.com/annadmitrieva/NLP-stuff/blob/master/domain_model.ipynb

In [0]:
!wget https://www.dropbox.com/s/lyfk0ur0pqvcgqv/domain_model.pkl?dl=0 -O domain_model.pkl
clear_output()

In [0]:
!wget https://www.dropbox.com/s/y35dm5chuv54psg/suggestions.zip?dl=0 -O suggestions.zip
clear_output()

In [0]:
!unzip suggestions.zip
clear_output()

In [0]:
import pandas as pd
import re
import nltk
import gensim
import joblib
from gensim.models import Word2Vec
from natasha import NamesExtractor, AddressExtractor, DatesExtractor

model = gensim.models.KeyedVectors.load_word2vec_format(
    'ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz', binary=False)
domain_model = joblib.load('domain_model.pkl')

In [0]:
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('punkt')
clear_output()

In [0]:
def domain(string):
    domain = domain_model.predict(list(string))[0]
    df = pd.read_csv(f'suggestions_{domain}.csv')
    return df

#Поиск неправильных сочетаний
В предыдущей версии этого алгоритма (https://github.com/annadmitrieva/NLP-stuff/blob/master/collocation_generation-Copy4.ipynb) в список плохих коллокаций попадали все встретившиеся в тексте биграммы из существительных или глаголов, которых не было в эталонном списке. В результате отлавливалось много пар слов, вообще не являющихся словосочетаниями (пример – "работы разрешение"). Решение: добавить синтаксический парсер. <br>
Будем считать плохими коллокациями такие пары "вершина-зависимое", которые состоят из существительных и глаголов и которых нет в эталонных списках правильных словосочетаний (для каждого домена текста – свой список).<br>
Эталонные списки коллокаций: https://drive.google.com/drive/folders/1k_N-DZ-nLL5ro66-LxIaE4-dRwirdwZh <br>
Код, которых использовался при сборе эталонных списков: https://github.com/MariaFjodorowa/catandthekittens/tree/develop/collocations/collocation_frequencies

Несколько плохих текстов, на которых будем проверять, насколько хорошо находятся плохие коллокации. Источник – примеры плохой лексической сочетаемости из домашнего задания по академическому письму 1 курса ОП "Фундаментальная и прикладная лингвистика" НИУ ВШЭ 2017 года.

In [0]:
bad_text_pol = """В то же время, Путин умолчал от египетского населения данные о том, 
насколько милитаризируется сама Россия и насколько выросли ее военные расходы. 
Обама улетел из Вашингтона на вертолете, а Байден - на поезде."""
bad_text_hist = """Маккензи ограничивается рамками исследования 1928 – 1943, исследуя преимущественно идеологию Коминтерна сталинской эпохи, 
перечисляет причины на то, чтобы рассматривать 1928 год как переломный. 
Александр Шубин посвящает свою книгу большей частью на исследование общественно-политических движений в послесталинский СССР, 
1953 – 85 годов. В строках читаются напряжение и недоверие власти."""
bad_text_ling = """Следует избегать наводящие вопросы. Тем не менее существовало несколько точек зрения о создании алфавита. 
Предлагая ту или иную гипотезу, нельзя основываться и ссылаться только на результаты своего собственного исследования. 
Необходимо внимательное изучение опыта других ученых. 
Термин морфология обычно соотносится к части грамматики языка. Однако этот термин в истории науки употреблялся и в совсем ином значении."""

##Подход 1: MaltParser (медленный и плохо работающий)
Для POS-тэггинга использовался метод pos_tag из NLTK для русского языка.<br>
Для синтаксического парсинга использовался MaltParser из NLTK с предобученной моделью для русского языка. <br>
Источник модели: http://corpus.leeds.ac.uk/mocky/ <br>

In [0]:
!wget http://corpus.leeds.ac.uk/tools/russian.mco
!wget http://maltparser.org/dist/maltparser-1.8.1.zip
!unzip maltparser-1.8.1.zip
clear_output()

In [0]:
!export MALT_PARSER=$HOME/maltparser-1.8.1/
!export MALT_MODEL=$HOME/russian.mco

In [0]:
def pos_tag_rus(tokens):
    return nltk.pos_tag(tokens, lang='rus')

In [0]:
syntax_parser = nltk.parse.malt.MaltParser(
    'maltparser-1.8.1', 'russian.mco', tagger=pos_tag_rus)

In [18]:
# проверим, правильно ли работает парсер
sents = nltk.sent_tokenize(bad_text_pol)
nodes = syntax_parser.parse_one(sents[0].split()).nodes
for node in nodes.keys():
    head = nodes[node]['head']
    if head is not None and nodes[head]['word'] is not None:
        print(f"Head: {nodes[head]['word']} {nodes[head]['ctag']}. " +
              f"Dependent: {nodes[node]['word']} {nodes[node]['ctag']}.")

Head: то A-PRO=n. Dependent: В PR.
Head: то A-PRO=n. Dependent: же PART.
Head: то A-PRO=n. Dependent: время, S.
Head: время, S. Dependent: Путин S.
Head: от PR. Dependent: умолчал V.
Head: Путин S. Dependent: от PR.
Head: населения S. Dependent: египетского A=n.
Head: то A-PRO=n. Dependent: населения S.
Head: населения S. Dependent: данные S.
Head: том, S. Dependent: насколько ADV.
Head: сама A-PRO=f. Dependent: милитаризируется V.
Head: насколько ADV. Dependent: и CONJ.
Head: выросли V. Dependent: насколько ADV.
Head: выросли V. Dependent: ее S.
Head: ее S. Dependent: военные A=pl.
Head: военные A=pl. Dependent: расходы. S.


In [0]:
def search(text, collocations):
    """text – строка, collocations – датафрейм pandas.
    Возвращается словарь candidates,
    где ключи – пары слов, а значения – пары тэгов"""
    candidates = {}
    first = list(collocations.first_word)
    second = list(collocations.second_word)
    clean_text = re.sub("\([a-zA-z\d .,:]*\)|[a-zA-z\d]*", "", text)
    # чистим текст от библиографических ссылок
    # и токенов, состоящих только из латиницы и цифр
    extractors = [NamesExtractor(), AddressExtractor(), DatesExtractor()]
    named_entities = []
    # составляем список всех именованных сущностей, встречающихся в тексте
    for extractor in extractors:
        matches = extractor(text)
        for match in matches:
            start, stop = match.span
            for i in re.findall('\w+', text[start: stop]):
                named_entities.append(i.lower())
    # разбиваем текст на предложения, а предложения на слова
    sents = nltk.sent_tokenize(text)
    for i in range(len(sents)):
        nodes = syntax_parser.parse_one(sents[i].split()).nodes
        for node in nodes.keys():
            head = nodes[node]['head']
            if head is not None and nodes[head]['word'] is not None \
                    and nodes[head]['word'].lower() not in named_entities \
                    and nodes[node]['word'].lower() not in named_entities \
                    and nodes[head]['ctag'] in ['S', 'V'] \
                    and nodes[node]['ctag'] in ['S', 'V']:
                word_1 = nodes[head]['word'].lower()
                word_2 = nodes[node]['word'].lower()
                tag_1 = nodes[head]['ctag']
                tag_2 = nodes[node]['ctag']
                if word_1 in first or word_2 in second:
                    if word_1 in first and word_2 in second:
                        indices_1 = [i for i, x in enumerate(first)
                                     if x == word_1]
                        indices_2 = [i for i, x in enumerate(second)
                                     if x == word_2]
                        if not set(indices_1).isdisjoint(indices_2):
                            pass
                        else:
                            candidates[(word_1, word_2)] = (tag_1, tag_2)
                    else:
                        candidates[(word_1, word_2)] = (tag_1, tag_2)
    return candidates

Проверим, как функция search работает на наших плохих текстах.

In [20]:
%%time
candidates_pol = search(bad_text_pol, pd.read_csv('suggestions_pol.csv'))

CPU times: user 467 ms, sys: 62 ms, total: 529 ms
Wall time: 10.8 s


In [21]:
candidates_pol

{('населения', 'данные'): ('S', 'S')}

In [22]:
%%time
candidates_hist = search(bad_text_hist, pd.read_csv('suggestions_hist.csv'))

CPU times: user 252 ms, sys: 66 ms, total: 318 ms
Wall time: 16.3 s


In [23]:
candidates_hist

{('недоверие', 'власти.'): ('S', 'S'),
 ('ограничивается', 'рамками'): ('V', 'S'),
 ('перечисляет', 'причины'): ('V', 'S'),
 ('рамками', 'исследования'): ('S', 'S')}

In [24]:
%%time
candidates_ling = search(bad_text_ling, pd.read_csv('suggestions_ling.csv'))

CPU times: user 448 ms, sys: 142 ms, total: 590 ms
Wall time: 31.7 s


In [25]:
candidates_ling

{('зрения', 'создании'): ('S', 'S'),
 ('изучение', 'опыта'): ('S', 'S'),
 ('наводящие', 'избегать'): ('V', 'V'),
 ('наводящие', 'следует'): ('V', 'V'),
 ('создании', 'алфавита.'): ('S', 'S'),
 ('термин', 'морфология'): ('S', 'S')}

Результаты не очень хорошие – выделенные пары либо вообще не являются словосочетаниями ("населения данные", "зрения создании"), либо являются правильными словосочетаниями ("недоверие власти", "термин морфология"), при этом ни одно из спорных словосочетаний ("умолчал данные", "избегать вопросы") не выделены. Во многом эти ошибки вызваны плохой работой синтаксического парсера. <br>
Кроме того, MaltParser довольно медленный.

## Подход 2: StanfordNLP
Попробуем другой парсер зависимостей. <br>
Источник: https://github.com/stanfordnlp/stanfordnlp. <br>
Зависимостный парсер в StanfordNLP работает лучше и быстрее (см. код ниже) MaltParser, кроме того, в него уже встроен морфологический тэггер с тэгом PROPN, поэтому не нужно искать какой-то ещё тэггер или дополнительно убирать именованные сущности.

In [0]:
!pip install stanfordnlp
clear_output()

In [0]:
import stanfordnlp
stanfordnlp.download('ru')
clear_output()

In [0]:
nlp = stanfordnlp.Pipeline(lang='ru', models_dir='/root/stanfordnlp_resources')
clear_output()

In [0]:
doc = nlp(bad_text_pol)

In [30]:
# посмотрим, как работает этот парсер
doc.sentences[0].dependencies[0:3]

[(<Word index=4;text=время;lemma=время;upos=NOUN;xpos=_;feats=Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing;governor=7;dependency_relation=parataxis>,
  'case',
  <Word index=1;text=В;lemma=в;upos=ADP;xpos=_;feats=_;governor=4;dependency_relation=case>),
 (<Word index=4;text=время;lemma=время;upos=NOUN;xpos=_;feats=Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing;governor=7;dependency_relation=parataxis>,
  'det',
  <Word index=2;text=то;lemma=тот;upos=DET;xpos=_;feats=Case=Acc|Gender=Neut|Number=Sing;governor=4;dependency_relation=det>),
 (<Word index=2;text=то;lemma=тот;upos=DET;xpos=_;feats=Case=Acc|Gender=Neut|Number=Sing;governor=4;dependency_relation=det>,
  'discourse',
  <Word index=3;text=же;lemma=же;upos=PART;xpos=_;feats=_;governor=2;dependency_relation=discourse>)]

In [0]:
def search(text, collocations):
    """text – строка, collocations – датафрейм pandas.
    Возвращается словарь candidates,
    где ключи – пары слов, а значения – пары тэгов"""
    candidates = {}
    first = list(collocations.first_word)
    second = list(collocations.second_word)
    clean_text = re.sub("\([a-zA-z\d .,:]*\)|[a-zA-z\d]*", "", text)
    # чистим текст от библиографических ссылок
    # и токенов, состоящих только из латиницы и цифр
    parsed_text = nlp(clean_text)
    for sent in parsed_text.sentences:
        for dep in sent.dependencies:
            if dep[0].upos in ['NOUN', 'VERB'] \
                    and dep[2].upos in ['NOUN', 'VERB']:
                word_1 = dep[0].text.lower()
                word_2 = dep[2].text.lower()
                tag_1 = dep[0].upos
                tag_2 = dep[2].upos
                if word_1 in first or word_2 in second:
                    if word_1 in first and word_2 in second:
                        indices_1 = [i for i, x in enumerate(first)
                                     if x == word_1]
                        indices_2 = [i for i, x in enumerate(second)
                                     if x == word_2]
                        if not set(indices_1).isdisjoint(indices_2):
                            pass
                        else:
                            candidates[(word_1, word_2)] = (tag_1, tag_2)
                    else:
                        candidates[(word_1, word_2)] = (tag_1, tag_2)
    return candidates

In [32]:
%%time
candidates_pol = search(bad_text_pol, pd.read_csv('suggestions_pol.csv'))

CPU times: user 132 ms, sys: 6 ms, total: 138 ms
Wall time: 141 ms


In [33]:
candidates_pol

{('выросли', 'расходы'): ('VERB', 'NOUN'),
 ('умолчал', 'время'): ('VERB', 'NOUN'),
 ('умолчал', 'данные'): ('VERB', 'NOUN'),
 ('умолчал', 'населения'): ('VERB', 'NOUN')}

In [34]:
%%time
candidates_hist = search(bad_text_hist, pd.read_csv('suggestions_hist.csv'))

CPU times: user 128 ms, sys: 3 ms, total: 131 ms
Wall time: 133 ms


In [35]:
candidates_hist

{('исследование', 'движений'): ('NOUN', 'NOUN'),
 ('недоверие', 'власти'): ('NOUN', 'NOUN'),
 ('ограничивается', 'маккензи'): ('VERB', 'NOUN'),
 ('ограничивается', 'перечисляет'): ('VERB', 'VERB'),
 ('ограничивается', 'рамками'): ('VERB', 'NOUN'),
 ('перечисляет', 'причины'): ('VERB', 'NOUN'),
 ('посвящает', 'книгу'): ('VERB', 'NOUN'),
 ('посвящает', 'частью'): ('VERB', 'NOUN'),
 ('рамками', 'исследования'): ('NOUN', 'NOUN'),
 ('рассматривать', 'год'): ('VERB', 'NOUN'),
 ('частью', 'исследование'): ('NOUN', 'NOUN')}

In [36]:
%%time
candidates_ling = search(bad_text_ling, pd.read_csv('suggestions_ling.csv'))

CPU times: user 168 ms, sys: 2.01 ms, total: 170 ms
Wall time: 177 ms


In [37]:
candidates_ling

{('вопросы', 'наводящие'): ('NOUN', 'VERB'),
 ('грамматики', 'языка'): ('NOUN', 'NOUN'),
 ('избегать', 'вопросы'): ('VERB', 'NOUN'),
 ('изучение', 'опыта'): ('NOUN', 'NOUN'),
 ('опыта', 'ученых'): ('NOUN', 'NOUN'),
 ('предлагая', 'гипотезу'): ('VERB', 'NOUN'),
 ('следует', 'избегать'): ('VERB', 'VERB'),
 ('создании', 'алфавита'): ('NOUN', 'NOUN'),
 ('соотносится', 'термин'): ('VERB', 'NOUN'),
 ('соотносится', 'части'): ('VERB', 'NOUN'),
 ('ссылаться', 'результаты'): ('VERB', 'NOUN'),
 ('термин', 'истории'): ('NOUN', 'NOUN'),
 ('термин', 'морфология'): ('NOUN', 'NOUN'),
 ('точек', 'создании'): ('NOUN', 'NOUN'),
 ('употреблялся', 'значении'): ('VERB', 'NOUN'),
 ('употреблялся', 'термин'): ('VERB', 'NOUN'),
 ('части', 'грамматики'): ('NOUN', 'NOUN')}

Уже лучше! Все предложенные пары слов являются словосочетаниями (хотя функция *search()* всё ещё иногда ошибочно отлавливает правильные коллокации).